<a href="https://colab.research.google.com/github/ww-Jing/Algorithmic_Trading/blob/main/ma_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance 

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

In [ ]:
# 定義策略
def moving_average_strategy(data, short_window, long_window):
    # 計算短期和長期均線
    data['short_mavg'] = data['Adj Close'].rolling(window=short_window).mean()
    data['long_mavg'] = data['Adj Close'].rolling(window=long_window).mean()
    # 產生交易訊號
    data['signal'] = np.where(data['short_mavg'] > data['long_mavg'], 1, 0)
    data['positions'] = data['signal'].diff().shift(1)
    
    return data

In [ ]:
# 載入歷史資料
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = '2022-03-07'
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# 設定均線期數和初始資金
short_window = 5
long_window = 100
initial_capital = 100000

# 執行回測策略
data_0 = moving_average_strategy(data, short_window, long_window)

In [ ]:
#計算收益
#方法1
data_1 = data_0.reset_index()
#取出有做買賣的日子
data_1['positions'] = data_1['positions'].replace(np.nan, 0)
data_1 = data_1[data_1['positions'] != 0][['Date','Adj Close', 'Close', 'positions']]

data_1['price'] = data_1['Adj Close']*data_1['positions']
data_1['ret'] = -(data_1['price']+data_1['price'].shift(1)) / data_1['price'].shift(1)
data_1['ret'] = data_1['ret'].dropna()
data_1['total_ret'] = (1 + data_1['ret']).cumprod()

#data_1.head()

,Date,Adj Close,Close,positions,price,ret,total_ret
100,2015-05-28,29.851091,32.945000,1.0,29.851091,NaN,NaN
123,2015-06-30,28.412676,31.357500,-1.0,-28.412676,-0.048186,0.951814
136,2015-07-20,29.916777,33.017502,1.0,29.916777,0.052938,1.002200
142,2015-07-28,27.948301,30.844999,-1.0,-27.948301,-0.065798,0.936257
209,2015-10-30,27.191927,29.875000,1.0,27.191927,-0.027063,0.910919


In [ ]:
#方法2 (方法跟法1一樣，但這裡利用pct_change.to_numpy去做運算)
data_1['pct_change'] = data_1['Adj Close'].pct_change()
pct_change = data_1['pct_change'].to_numpy()
pct_change = np.delete(pct_change, [0])

total_ret = 1 

for i in range(len(pct_change)):
  total_ret *= 1 + pct_change[i]

final_rate = pow(total_ret, 1/(len(data_1['positions'])))-1
print(total_ret)
print(final_rate)

4.369812241331291
0.045702058487023134
